In [1]:
import pandas as pd
import numpy as np
import json
import math

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import spearmanr

In [2]:
vignette = json.load(open("./results/results_vignette.json","r"))
convo = json.load(open("./results/results_conversation.json","r"))
convo_without_pe = json.load(open("./results/results_conversation_withoutPE.json","r"))

In [3]:
public_cases = ["public_case0"+str(x) for x in range(1,10)] + ["public_case"+str(x) for x in range(10,101)]
private_cases = ["private_case0"+str(x) for x in range(1,10)] + ["private_case"+str(x) for x in range(10,41)]

In [4]:
temp = []
for case in public_cases:
    temp.append([x[0] for x in vignette["gpt-3.5"][case]["frq"]["autoeval_raw"]])
vignette_gpt3 = pd.DataFrame(temp, index=public_cases).T

temp = []
for case in public_cases:
    temp.append([x[0] for x in vignette["gpt-4"][case]["frq"]["autoeval_raw"]])
vignette_gpt4 = pd.DataFrame(temp, index=public_cases).T

In [5]:
temp = []
for case in public_cases:
    temp.append([x[0] for x in convo["gpt-3.5"]["multi-turn"][case]["frq"]["autoeval_raw"]])
convo_gpt3 = pd.DataFrame(temp, index=public_cases).T

temp = []
for case in public_cases:
    temp.append([x[0] for x in convo["gpt-4"]["multi-turn"][case]["frq"]["autoeval_raw"]])
convo_gpt4 = pd.DataFrame(temp, index=public_cases).T

In [6]:
temp = []
for case in public_cases:
    temp.append([x[0] for x in convo_without_pe["gpt-3.5"][case]["frq"]["autoeval_raw"]])
convo_without_pe_gpt3 = pd.DataFrame(temp, index=public_cases).T

temp = []
for case in public_cases:
    temp.append([x[0] for x in convo_without_pe["gpt-4"][case]["frq"]["autoeval_raw"]])
convo_without_pe_gpt4 = pd.DataFrame(temp, index=public_cases).T

In [7]:
public_auto = pd.DataFrame([vignette_gpt3.mean(), vignette_gpt4.mean(),
                            convo_gpt3.mean(), convo_gpt4.mean(),
                            convo_without_pe_gpt3.mean(), convo_without_pe_gpt4.mean()], 
                             index=["vignette_FRQ_GPT3", "vignette_FRQ_GPT4", 
                                    "conversation_PE_FRQ_GPT3", "conversation_PE_FRQ_GPT4",
                                    "conversation_FRQ_GPT3", "conversation_FRQ_GPT4"]).T

In [8]:
derm_annots = pd.read_csv("./results/expert_annotations/dermatologist_annotations_publicdata_graderai_accuracy_assessment.tsv",sep="\t",index_col=0)
for col in public_auto.columns:
    temp = []
    for case in public_cases:
        if math.isnan(derm_annots["ben_" + col][case]):
            temp.append(derm_annots[col][case])
        else:
            temp.append(derm_annots["ben_" + col][case])
            
    derm_annots[col] = temp

In [9]:
for col in public_auto.columns:
    x = public_auto[col]
    y = derm_annots[col]
    print(col, spearmanr(x,y))

vignette_FRQ_GPT3 SignificanceResult(statistic=0.9362758819676327, pvalue=2.5438324681228514e-46)
vignette_FRQ_GPT4 SignificanceResult(statistic=0.9547371626358471, pvalue=2.087333522393674e-53)
conversation_PE_FRQ_GPT3 SignificanceResult(statistic=0.7289224200246791, pvalue=8.154032957778248e-18)
conversation_PE_FRQ_GPT4 SignificanceResult(statistic=0.9426964337417999, pvalue=1.6332324533526964e-48)
conversation_FRQ_GPT3 SignificanceResult(statistic=0.8894146773863807, pvalue=4.327020868135299e-35)
conversation_FRQ_GPT4 SignificanceResult(statistic=0.9249774004668274, pvalue=5.749464516752942e-43)
